In [1]:
import numpy as np 
import pandas as pd
import pickle
import tensorflow as tf
from Bio import SeqIO
import os
from feature_extarction import *
from Utilis import *
from predictor import *
from sklearn.metrics import accuracy_score , f1_score , precision_score , recall_score , roc_auc_score , matthews_corrcoef , confusion_matrix,precision_recall_curve

2023-12-11 22:24:47.502545: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-11 22:24:48.932955: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.config.LogicalDevice(name='/physical_device:GPU:0', device_type='GPU')
tf.config.PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

In [3]:
tf.sysconfig.get_build_info() 

OrderedDict([('cpu_compiler', '/usr/lib/llvm-16/bin/clang'),
             ('cuda_compute_capabilities',
              ['sm_35', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']),
             ('cuda_version', '11.8'),
             ('cudnn_version', '8'),
             ('is_cuda_build', True),
             ('is_rocm_build', False),
             ('is_tensorrt_build', True)])

In [4]:
file = 'Data/Train_335.fa'

sequences_train = [seq for seq in SeqIO.parse(file,'fasta')]

In [5]:
file = 'test/Test_60.fa'

sequences_test = [seq for seq in SeqIO.parse(file,'fasta')]

In [6]:
file = 'test/Test_315.fa'

sequences_test_315 = [seq for seq in SeqIO.parse(file,'fasta')]

In [6]:
sequences_test[0].seq

Seq('SLRQEDFPPRIVEHPSDLIVSKGEPATLNCKAEGRPTPTIEWYKGGERVETDKD...000')

In [6]:
ProtName_train = []
y_train = []
seqs_train = []


for seq in sequences_train:
    ProtName_train.append(seq.name)
    y = []
    sequence = []
    for i in seq.seq:
        if  i.isdigit() == False:
            sequence.append(i)
        else:
            y.append(int(i))
    

    y_train.append(np.array(y))
    seqs_train.append(sequence)

In [7]:
ProtName_test = []
y_test = []
seqs_test = []


for seq in sequences_test:
    ProtName_test.append(seq.name)
    y = []
    sequence = []
    for i in seq.seq:
        if  i.isdigit() == False:
            sequence.append(i)
        else:
            y.append(int(i))
    

    y_test.append(y)
    seqs_test.append(sequence)

In [9]:
ProtName_test_315 = []
y_test_315 = []
seqs_test_315 = []


for seq in sequences_test_315:
    ProtName_test_315.append(seq.name)
    y = []
    sequence = []
    for i in seq.seq:
        if  i.isdigit() == False:
            sequence.append(i)
        else:
            y.append(int(i))
    

    y_test_315.append(y)
    seqs_test_315.append(sequence)

In [87]:
ProtName_train.index('4iu2B')

1

In [88]:
print(seqs_train[0])
print(y_train[0])

['P', 'A', 'A', 'K', 'S', 'I', 'V', 'T', 'L', 'D', 'V', 'K', 'P', 'W', 'D', 'D', 'E', 'T', 'N', 'L', 'E', 'E', 'M', 'V', 'A', 'N', 'V', 'K', 'A', 'I', 'E', 'M', 'E', 'G', 'L', 'T', 'W', 'G', 'A', 'H', 'Q', 'F', 'I', 'P', 'I', 'G', 'F', 'G', 'I', 'K', 'K', 'L', 'Q', 'I', 'N', 'C', 'V', 'V', 'E', 'D', 'D', 'K', 'V', 'S', 'L', 'D', 'D', 'L', 'Q', 'Q', 'S', 'I', 'E', 'E', 'D', 'E', 'D', 'H', 'V', 'Q', 'S', 'T', 'D', 'I', 'A', 'A', 'M', 'Q', 'K', 'L']
[0 0 1 1 1 1 0 1 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 0 1 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 0 1 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1]


In [89]:
print(seqs_test[0])
print(y_test[0])

['S', 'L', 'R', 'Q', 'E', 'D', 'F', 'P', 'P', 'R', 'I', 'V', 'E', 'H', 'P', 'S', 'D', 'L', 'I', 'V', 'S', 'K', 'G', 'E', 'P', 'A', 'T', 'L', 'N', 'C', 'K', 'A', 'E', 'G', 'R', 'P', 'T', 'P', 'T', 'I', 'E', 'W', 'Y', 'K', 'G', 'G', 'E', 'R', 'V', 'E', 'T', 'D', 'K', 'D', 'D', 'P', 'R', 'S', 'H', 'R', 'M', 'L', 'L', 'P', 'S', 'G', 'S', 'L', 'F', 'F', 'L', 'R', 'I', 'V', 'H', 'G', 'R', 'K', 'S', 'R', 'P', 'D', 'E', 'G', 'V', 'Y', 'V', 'C', 'V', 'A', 'R', 'N', 'Y', 'L', 'G', 'E', 'A', 'V', 'S', 'H', 'D', 'A', 'S', 'L', 'E', 'V', 'A']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
print(ProtName_train[0][:-1])
train_data_path = './Data/'

1f60


In [91]:
os.system("wget -P {} http://www.rcsb.org/pdb/files/{}.pdb.gz".format(train_data_path, ProtName_train[0][:-1]))

URL transformed to HTTPS due to an HSTS policy
--2023-11-09 23:41:18--  https://www.rcsb.org/pdb/files/1f60.pdb.gz
Resolving www.rcsb.org (www.rcsb.org)... 128.6.159.248
Connecting to www.rcsb.org (www.rcsb.org)|128.6.159.248|:443... 

connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.rcsb.org/download/1f60.pdb.gz [following]
--2023-11-09 23:41:19--  https://files.rcsb.org/download/1f60.pdb.gz
Resolving files.rcsb.org (files.rcsb.org)... 128.6.159.100
Connecting to files.rcsb.org (files.rcsb.org)|128.6.159.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103657 (101K) [application/octet-stream]
Saving to: ‘./Data/1f60.pdb.gz.1’

     0K .......... .......... .......... .......... .......... 49%  135K 0s
    50K .......... .......... .......... .......... .......... 98%  354K 0s
   100K .                                                     100% 23.7M=0.5s

2023-11-09 23:41:21 (198 KB/s) - ‘./Data/1f60.pdb.gz.1’ saved [103657/103657]



0

In [92]:
for prot in ProtName_train:
    if os.path.exists(train_data_path + "{}.pdb.gz".format(prot[:-1])) == False:
        os.system("wget -P {} http://www.rcsb.org/pdb/files/{}.pdb.gz".format(train_data_path, prot[:-1]))

In [93]:
for prot in ProtName_train:
    pdb_extract(train_data_path, prot[:-1],prot[-1])

URL transformed to HTTPS due to an HSTS policy
--2023-11-09 23:41:21--  https://www.rcsb.org/pdb/files/1f60.pdb.gz
Resolving www.rcsb.org (www.rcsb.org)... 128.6.159.248
Connecting to www.rcsb.org (www.rcsb.org)|128.6.159.248|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.rcsb.org/download/1f60.pdb.gz [following]
--2023-11-09 23:41:22--  https://files.rcsb.org/download/1f60.pdb.gz
Resolving files.rcsb.org (files.rcsb.org)... 128.6.159.100
Connecting to files.rcsb.org (files.rcsb.org)|128.6.159.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103657 (101K) [application/octet-stream]
Saving to: ‘./data_ext/1f60.pdb.gz’

     0K .......... .......... .......... .......... .......... 49% 98.7K 1s
    50K .......... .......... .......... .......... .......... 98%  179K 0s
   100K .                                                     100% 2.29T=0.8s

2023-11-09 23:41:24 (129 KB/s) - ‘./data_ext/1f60.pdb.gz

In [8]:
#Add path here accoding to your device
data_path='./Data/' #output folder path for features
Software_path = "./"    
dssp = Software_path + "dssp-3.1.4/mkdssp" #dssp software folder path for features extraction
PSIBLAST = Software_path + "Blastx/bin/psiblast" #PSIBLAST software folder path for features extraction
HHBLITS = Software_path + "hh-suite/build/bin/hhblits" #HH-SUITE software folder path for features extraction
UR90 = "./unirefdb/uniref90.fasta" #database for pssm path #PSIBLAST DATABASE folder path for features extraction
HHDB = "./hmmDB/uniclust30_2017_10" #database hmm path #HH-SUITE DATABASE folder path for features extraction
pre_path='./Feature/'

In [ ]:
for prot,sequence in zip(ProtName_train,seqs_train):
    get_dssp(prot,sequence,'./Data/')

In [ ]:
for prot in ProtName_train:
    prepare_hhm(prot)

In [ ]:
count = 0
for prot in ProtName_train:
    prepare_pssm(prot)
    count += 1
    print(count)

1


2


3


In [11]:
X_train_1 = []

path=pre_path
with open(f'{path}/data_seq/data_ID.dat','rb') as FH:
    data0 =pickle.load(FH)
    
    
pssm=np.load(f'{path}pssm/pssm.npy',allow_pickle=True)
dssp=np.load(f'{path}dssp/dssp.npy',allow_pickle=True)
hhm=np.load(f'{path}hhm/hhm.npy',allow_pickle=True)

for prot in ProtName_train:
    
    i_0=0
    for index_0 in data0[:,0]:  
        if prot.lower()==index_0.lower():
            found=1
            seq=data0[i_0,1]
            tEMP_0=pssm[1,i_0]
            tEMP_1=dssp[1,i_0]
            tEMP_2=hhm[1,i_0]
            w_size = 19

            f1 = windowing(tEMP_0, seq, w_size, 0, 20)
            f2 = windowing(tEMP_1, seq, w_size, 0, 14)
            f3 = windowing(tEMP_2, seq, w_size, 0, 20)

            if not X_train_1:
                # If X_train is empty, just append the matrices
                X_train_1.extend([f1, f2, f3])
            else:
                # Use numpy.concatenate to append along the first dimension
                X_train_1[0] = np.concatenate([X_train_1[0], f1])
                X_train_1[1] = np.concatenate([X_train_1[1], f2])
                X_train_1[2] = np.concatenate([X_train_1[2], f3])
            break
        i_0=i_0+1

X_train_1[0].shape

(66276, 19, 20)

In [9]:
X_train_1 = []
for prot, sequence in zip(ProtName_train, seqs_train):
    tEMP_0 = np.load(f'{data_path}pssm/{prot}.npy')
    tEMP_1 = np.load(f'{data_path}dssp/{prot}.npy')
    tEMP_2 = np.load(f'{data_path}hmm/{prot}.npy')

    w_size = 19

    f1 = windowing(tEMP_0, sequence, w_size, 0, 20)
    f2 = windowing(tEMP_1, sequence, w_size, 0, 14)
    f3 = windowing(tEMP_2, sequence, w_size, 0, 20)

    if not X_train_1:
        # If X_train is empty, just append the matrices
        X_train_1.extend([f1, f2, f3])
    else:
        # Use numpy.concatenate to append along the first dimension
        X_train_1[0] = np.concatenate([X_train_1[0], f1])
        X_train_1[1] = np.concatenate([X_train_1[1], f2])
        X_train_1[2] = np.concatenate([X_train_1[2], f3])

X_train_1[0].shape


(66366, 19, 20)

In [10]:
y_train_1 = np.concatenate(y_train)
y_train_1.shape

(66366,)

In [14]:
y_train_1.sum()/y_train_1.shape[0]

0.15631498056233614

In [31]:
import numpy as np

def balance_data(X, y, target_ratio=0.6, random_seed=42):
    # Set the random seed for reproducibility
    if random_seed is not None:
        np.random.seed(random_seed)

    # Find the indices of positive (1) class samples
    positive_indices = np.where(y == 1)[0]
    negative_indices = np.where(y == 0)[0]

    # Calculate the desired number of positive samples to duplicate
    num_positive_samples = len(positive_indices)
    num_samples_to_duplicate = abs(int(num_positive_samples * (target_ratio)))


    duplicated_positive_indices = np.random.choice(positive_indices, num_samples_to_duplicate, replace=True)

    e = duplicated_positive_indices
    

    # Combine the original positive samples with the duplicated ones and the negative samples
    balanced_indices = np.concatenate([positive_indices, duplicated_positive_indices, negative_indices])

    # Shuffle the indices to mix the order of samples
    np.random.shuffle(balanced_indices)

    # Create balanced X and y arrays
    X_balanced0 = X[0][balanced_indices]
    X_balanced1 = X[1][balanced_indices]
    X_balanced2 = X[2][balanced_indices]

    X_balanced = [X_balanced0, X_balanced1, X_balanced2]
    y_balanced = y[balanced_indices]

    return X_balanced, y_balanced

# Example usage:
X_train_balanced, y_train_balanced = balance_data(X_train_1, y_train_1, target_ratio=3, random_seed=42)




In [11]:
def balance_data(X, y, target_ratio=0.6, random_seed=42):
    # Set the random seed for reproducibility
    if random_seed is not None:
        np.random.seed(random_seed)
    
    # Find the indices of positive (1) and negative (0) class samples
    positive_indices = np.where(y == 1)[0]
    negative_indices = np.where(y == 0)[0]

    # Calculate the desired number of negative samples
    num_positive_samples = len(positive_indices)
    num_negative_samples = int(num_positive_samples / target_ratio - num_positive_samples)

    # Randomly select a subset of negative samples to keep the desired ratio
    random_negative_indices = np.random.choice(negative_indices, num_negative_samples, replace=False)

    # Combine the selected positive and negative samples
    balanced_indices = np.concatenate([positive_indices, random_negative_indices])

    # Shuffle the indices to mix the order of samples
    np.random.shuffle(balanced_indices)

    # Create balanced X and y arrays
    X_balanced0 = X[0][balanced_indices]
    X_balanced1 = X[1][balanced_indices]
    X_balanced2 = X[2][balanced_indices]

    X_balanced = [X_balanced0, X_balanced1, X_balanced2]
    y_balanced = y[balanced_indices]

    return X_balanced, y_balanced


In [45]:
y_train_balanced.shape

(34580,)

In [12]:
X_test = []
for prot,sequence in zip(ProtName_test,seqs_test):
    tEMP_0=np.load(f'{data_path}pssm/{prot}.npy')
    tEMP_1=np.load(f'{data_path}dssp/{prot}.npy')
    tEMP_2=np.load(f'{data_path}hmm/{prot}.npy')

    w_size=19
    
    f1=windowing(tEMP_0,sequence,w_size,0,20)
    f2=windowing(tEMP_1,sequence,w_size,0,14)
    f3=windowing(tEMP_2,sequence,w_size,0,20)

    if not X_test:
        # If X_train is empty, just append the matrices
        X_test.extend([f1, f2, f3])
    else:
        # Use numpy.concatenate to append along the first dimension
        X_test[0] = np.concatenate([X_test[0], f1])
        X_test[1] = np.concatenate([X_test[1], f2])
        X_test[2] = np.concatenate([X_test[2], f3])

    X_test[0].shape

In [13]:
X_test[0].shape

(13144, 19, 20)

In [14]:
y_test = np.concatenate(y_test)
y_test.shape

(13144,)

In [ ]:
X_test_315 = []
for prot,sequence in zip(ProtName_test_315,seqs_test_315):
    tEMP_0=np.load(f'{data_path}pssm/{prot}.npy')
    tEMP_1=np.load(f'{data_path}dssp/{prot}.npy')
    tEMP_2=np.load(f'{data_path}hmm/{prot}.npy')

    w_size=19
    
    f1=windowing(tEMP_0,sequence,w_size,0,20)
    f2=windowing(tEMP_1,sequence,w_size,0,14)
    f3=windowing(tEMP_2,sequence,w_size,0,20)

    if not X_test_315:
        # If X_train is empty, just append the matrices
        X_test_315.extend([f1, f2, f3])
    else:
        # Use numpy.concatenate to append along the first dimension
        X_test_315[0] = np.concatenate([X_test_315[0], f1])
        X_test_315[1] = np.concatenate([X_test_315[1], f2])
        X_test_315[2] = np.concatenate([X_test_315[2], f3])

print(X_test_315[0].shape)
    #y_test_315 = np.concatenate(y_test_315)
print(y_test_315.shape)


NameError: name 'ProtName_test_315' is not defined

In [15]:
X_train = []
for prot,sequence in zip(ProtName_train,seqs_train):
    tEMP_0=np.load(f'{data_path}pssm/{prot}.npy')
    tEMP_1=np.load(f'{data_path}dssp/{prot}.npy')
    tEMP_2=np.load(f'{data_path}hmm/{prot}.npy')

    w_size=19
    
    f1=windowing(tEMP_0,sequence,w_size,0,20)
    f2=windowing(tEMP_1,sequence,w_size,0,14)
    f3=windowing(tEMP_2,sequence,w_size,0,20)
    X_train.append([f1,f2,f3])

In [17]:
X_train[0][0].shape

(90, 19, 20)

In [ ]:
X_test = []
for prot,sequence in zip(ProtName_test,seqs_test):
    tEMP_0=np.load(f'{data_path}pssm/{prot}.npy')
    tEMP_1=np.load(f'{data_path}dssp/{prot}.npy')
    tEMP_2=np.load(f'{data_path}hmm/{prot}.npy')

    w_size=19
    
    f1=windowing(tEMP_0,sequence,w_size,0,20)
    f2=windowing(tEMP_1,sequence,w_size,0,14)
    f3=windowing(tEMP_2,sequence,w_size,0,20)
    X_test.append([f1,f2,f3])

In [ ]:
X_train = np.array(X_train)

/tmp/ipykernel_12746/1757705326.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array(X_train)


In [15]:
# Load the TensorFlow model
tf_model = tf.keras.models.load_model('model/model_19.h5')
ppi_model2 = tf.keras.models.load_model('model/ppi_model9.h5')

2023-12-11 22:26:04.567149: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-11 22:26:04.907973: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-11 22:26:04.908344: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [16]:
tf_model.summary()

Model: "functional_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_16 (InputLayer)       [(None, 19, 20)]             0         []                            
                                                                                                  
 input_17 (InputLayer)       [(None, 19, 14)]             0         []                            
                                                                                                  
 input_18 (InputLayer)       [(None, 19, 20)]             0         []                            
                                                                                                  
 conv1d_30 (Conv1D)          (None, 15, 64)               6464      ['input_16[0][0]']            
                                                                                      

In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Create your model
def PPI_Model():
    # Input layers
    input1 = tf.keras.layers.Input(shape=(19, 20), name='input_16')
    input2 = tf.keras.layers.Input(shape=(19, 14), name='input_17')
    input3 = tf.keras.layers.Input(shape=(19, 20), name='input_18')

    # CNN layers with ReLU activation
    conv1d_30 = layers.Conv1D(64, 5, activation='relu', name='conv1d_30')(input1)
    conv1d_33 = layers.Conv1D(64, 5, activation='relu', name='conv1d_33')(input2)
    conv1d_36 = layers.Conv1D(64, 5, activation='relu', name='conv1d_36')(input3)

    conv1d_31 = layers.Conv1D(32, 3, activation='relu', name='conv1d_31')(conv1d_30)
    conv1d_34 = layers.Conv1D(32, 3, activation='relu', name='conv1d_34')(conv1d_33)
    conv1d_37 = layers.Conv1D(32, 3, activation='relu', name='conv1d_37')(conv1d_36)

    conv1d_32 = layers.Conv1D(16, 3, activation='relu', name='conv1d_32')(conv1d_31)
    conv1d_35 = layers.Conv1D(16, 3, activation='relu', name='conv1d_35')(conv1d_34)
    conv1d_38 = layers.Conv1D(16, 3, activation='relu', name='conv1d_38')(conv1d_37)

    # Layer normalization
    layer_normalization_10 = layers.LayerNormalization(name='layer_normalization_10')(conv1d_32)
    layer_normalization_11 = layers.LayerNormalization(name='layer_normalization_11')(conv1d_35)
    layer_normalization_12 = layers.LayerNormalization(name='layer_normalization_12')(conv1d_38)

    # Dropout layers
    dropout_10 = layers.Dropout(0.1, name='dropout_10')(layer_normalization_10)
    dropout_11 = layers.Dropout(0.2, name='dropout_11')(layer_normalization_11)
    dropout_12 = layers.Dropout(0.3, name='dropout_12')(layer_normalization_12)

    # Concatenation
    concatenated = layers.Concatenate(name='concatenate_5')([dropout_10, dropout_11, dropout_12])

    # Flatten
    flattened = layers.Flatten(name='flatten_5')(concatenated)

    # Dense layers with ReLU activation
    dense_20 = layers.Dense(256, activation='relu', name='dense_20')(flattened)
    dense_21 = layers.Dense(128, activation='relu', name='dense_21')(dense_20)
    dense_22 = layers.Dense(64, activation='relu', name='dense_22')(dense_21)

    # Final layer with sigmoid activation
    output = layers.Dense(1, activation='sigmoid', name='dense_23')(dense_22)

    model = models.Model(inputs=[input1, input2, input3], outputs=output, name='PPI_Model')

    return model

# Instantiate the model
ppi_model = PPI_Model()

# Display model summary
ppi_model.summary()


Model: "PPI_Model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_16 (InputLayer)       [(None, 19, 20)]             0         []                            
                                                                                                  
 input_17 (InputLayer)       [(None, 19, 14)]             0         []                            
                                                                                                  
 input_18 (InputLayer)       [(None, 19, 20)]             0         []                            
                                                                                                  
 conv1d_30 (Conv1D)          (None, 15, 64)               6464      ['input_16[0][0]']            
                                                                                          

In [18]:
metrics = [tf.keras.metrics.AUC(curve='PR', name='prc'),
           'accuracy']



early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=15,
    mode='max',
    restore_best_weights=True)

In [92]:
# Compile the model
ppi_model.compile(optimizer=tf.keras.optimizers.experimental.SGD(learning_rate=0.001,weight_decay=0.01,momentum=0.9),  
                  loss='binary_crossentropy',  
                  metrics=metrics)  

In [49]:
ppi_model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='binary_crossentropy',
                  metrics=metrics)

In [79]:
ppi_model.compile(optimizer=tf.keras.optimizers.Lion(weight_decay=0.5,global_clipnorm=1.0),  
                  loss='binary_crossentropy',  
                  metrics=metrics)  

In [44]:
ppi_model.compile(optimizer=tf.keras.optimizers.experimental.Adagrad(),  # You can adjust the learning rate as needed
                  loss='binary_crossentropy',  
                  metrics=metrics)

In [ ]:
X_train_1[0][53093:].shape

(13273, 19, 20)

In [ ]:
xt = []
xt.append(X_train_1[0][:53092])
xt.append(X_train_1[1][:53092])
xt.append(X_train_1[2][:53092])

yt = y_train_1[:53092]

x_val = []
x_val.append(X_train_1[0][53093:])
x_val.append(X_train_1[1][53093:])
x_val.append(X_train_1[2][53093:])

y_val = y_train_1[53093:]

In [93]:
X_train_balanced, y_train_balanced = balance_data(X_train_1, y_train_1, target_ratio=0.5, random_seed=42)

In [94]:
y_train_balanced.shape

(20748,)

In [95]:
ppi_model.fit(X_train_balanced, y_train_balanced, epochs=30, batch_size=64, verbose=1)

Epoch 1/30
325/325 [==============================] - 5s 6ms/step - loss: 0.6989 - prc: 0.4664 - accuracy: 0.5165
Epoch 2/30
325/325 [==============================] - 2s 6ms/step - loss: 0.6785 - prc: 0.5822 - accuracy: 0.5689
Epoch 3/30
325/325 [==============================] - 2s 6ms/step - loss: 0.6617 - prc: 0.6279 - accuracy: 0.6016
Epoch 4/30
325/325 [==============================] - 2s 6ms/step - loss: 0.6496 - prc: 0.6438 - accuracy: 0.6200
Epoch 5/30
325/325 [==============================] - 2s 6ms/step - loss: 0.6391 - prc: 0.6664 - accuracy: 0.6309
Epoch 6/30
325/325 [==============================] - 2s 6ms/step - loss: 0.6355 - prc: 0.6733 - accuracy: 0.6361
Epoch 7/30
325/325 [==============================] - 2s 6ms/step - loss: 0.6268 - prc: 0.6869 - accuracy: 0.6456
Epoch 8/30
325/325 [==============================] - 2s 6ms/step - loss: 0.6182 - prc: 0.6978 - accuracy: 0.6543
Epoch 9/30
325/325 [==============================] - 2s 6ms/step - loss: 0.6114 - prc: 

In [ ]:
ppi_model.save('model/ppi_model9.h5')

/home/mahmoud/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [96]:
y_pred = ppi_model.predict(X_test)
y_pred = np.squeeze(y_pred)
y_pred.max()

411/411 [==============================] - 1s 2ms/step


0.9862191

In [26]:
y_pred = np.where(y_pred < 0.5, 0, 1)
y_pred.sum()

235

In [27]:
y_pred.sum()/y_pred.shape[0]

0.017878880097382838

In [68]:
y_test.sum()/ y_test.shape[0]

0.15786670724284846

In [66]:
y_test.sum()

2075

In [97]:
y_pred = ppi_model.predict(X_test)
y_pred = np.squeeze(y_pred)
y_pred = np.where(y_pred > 0.5, 1, 0)



print('Accuracy of ppi_model :',accuracy_score(y_test, y_pred))

print('F1 of ppi_model :',f1_score(y_test, y_pred))

print('Precision of ppi_model :',precision_score(y_test, y_pred))
129
print('Recall of ppi_model :',recall_score(y_test, y_pred))

print('MCC of ppi_model :',matthews_corrcoef(y_test, y_pred))

print('AUC of ppi_model :',roc_auc_score(y_test, y_pred))

411/411 [==============================] - 1s 2ms/step
Accuracy of ppi_model : 0.6088709677419355
F1 of ppi_model : 0.3585776668746101
Precision of ppi_model : 0.24191919191919192
Recall of ppi_model : 0.6925301204819277
MCC of ppi_model : 0.2093250640395229
AUC of ppi_model : 0.642859151848155


In [27]:
ppi_models = []
for i in range(0,31):
    X_train_balanced, y_train_balanced = balance_data(X_train_1, y_train_1, target_ratio=0.5, random_seed=i)
    ppi_model = PPI_Model()
    ppi_model.compile(optimizer=tf.keras.optimizers.experimental.Nadam(learning_rate=0.001,weight_decay=0.05,use_ema=True),
                  loss='binary_crossentropy',
                  metrics=metrics)
    
    ppi_model.fit(X_train_balanced, y_train_balanced, epochs=30, batch_size=64, verbose=0)

    y_pred = ppi_model.predict(X_test)
    y_pred = np.squeeze(y_pred)
    y_pred = np.where(y_pred > 0.5, 1, 0)
    mcc = matthews_corrcoef(y_test, y_pred)
    ppi_models.append((ppi_model, mcc,i))

411/411 [==============================] - 1s 2ms/step


In [25]:
ppi_models.sort(key=lambda x: x[1],reverse=True)

In [21]:
ppi_models[0][2]

7

In [26]:
model_exp = ppi_models[0][0]
y_pred = model_exp.predict(X_test)
y_pred = np.squeeze(y_pred)
y_pred = np.where(y_pred > 0.5, 1, 0)


print('Accuracy of ppi_model :',accuracy_score(y_test, y_pred))

print('F1 of ppi_model :',f1_score(y_test, y_pred))

print('Precision of ppi_model :',precision_score(y_test, y_pred))

print('Recall of ppi_model :',recall_score(y_test, y_pred))

print('MCC of ppi_model :',matthews_corrcoef(y_test, y_pred))

print('AUC of ppi_model :',roc_auc_score(y_test, y_pred))

411/411 [==============================] - 1s 2ms/step
Accuracy of ppi_model : 0.6715611685940353
F1 of ppi_model : 0.35480496188910476
Precision of ppi_model : 0.2571490467937608
Recall of ppi_model : 0.5720481927710843
MCC of ppi_model : 0.2003298256301579
AUC of ppi_model : 0.6311320555507783


In [ ]:
# Assuming X_pretraining is a list of three arrays and y_pretraining is a NumPy array
for x , y in zip(X_train,y_train):
    y = np.array(y)

    if x[0].shape[0] > 200:
        batch_size = 64
        eps = 19
    elif x[0].shape[0] > 100:
        batch_size = 32
        eps = 23
    else:
        batch_size = 16
        eps = 29

    ppi_model.fit(x, y, epochs=12, batch_size=batch_size,verbose=1)
